## Deduplicating the febrl3 dataset

See A.2  [here](https://arxiv.org/pdf/2008.04443.pdf) and [here](https://recordlinkage.readthedocs.io/en/latest/ref-datasets.html) for the source of this data

<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_examples_notebooks/docs/demos/examples/duckdb/febrl3.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_examples_notebooks

In [2]:
import pandas as pd
import altair as alt
from splink.datasets import splink_datasets

df = splink_datasets.febrl3
df = df.rename(columns=lambda x: x.strip())

df["cluster"] = df["rec_id"].apply(lambda x: "-".join(x.split('-')[:2]))
df.head(2)

,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,cluster
0,rec-1496-org,mitchell,green,7,wallaby place,delmar,cleveland,2119,sa,19560409,1804974,rec-1496
1,rec-552-dup-3,harley,mccarthy,177,pridhamstreet,milton,marsden,3165,nsw,19080419,6089216,rec-552


In [3]:
from splink.linker import Linker
from splink.database_api import DuckDBAPI

# TODO:  Allow missingness to be analysed without a linker

settings = {
    "unique_id_column_name": "rec_id",
    "link_type": "dedupe_only",
}

linker = Linker(df, settings, database_api=DuckDBAPI())

It's usually a good idea to perform exploratory analysis on your data so you understand what's in each column and how often it's missing:

In [4]:
linker.missingness_chart()

alt.LayerChart(...)

In [5]:
linker.profile_columns(["given_name", "surname"])
# Note: running linker.profile_columns() will profile all columns

alt.VConcatChart(...)

In [6]:
from splink.blocking_rule_library import block_on

blocking_rules = [
        block_on("soc_sec_id"),
        block_on("given_name"),
        block_on("surname"),
        block_on("date_of_birth"),
        block_on("postcode"),
]

# TODO: cumulative_num_comparisons_from_blocking_rules_chart should accept
# BlockingRuleCreator objects
blocking_rules_not_creator = [br.get_blocking_rule("duckdb") for br in blocking_rules]

linker.cumulative_num_comparisons_from_blocking_rules_chart(blocking_rules_not_creator)

alt.Chart(...)

In [7]:
from splink.linker import Linker
import splink.comparison_library as cl
import splink.comparison_template_library as ctl


settings = {
    "unique_id_column_name": "rec_id",
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": blocking_rules,
    "comparisons": [
        ctl.NameComparison("given_name").configure(term_frequency_adjustments=True),
        ctl.NameComparison("surname").configure(term_frequency_adjustments=True),
        # TODO: Fix date comparison
        # ctl.date_comparison("date_of_birth",
        #                     damerau_levenshtein_thresholds=[],
        #                     cast_strings_to_date=True,
        #                     invalid_dates_as_null=True,
        #                     date_format="%Y%m%d"),
        cl.LevenshteinAtThresholds("date_of_birth", [2]),
        cl.LevenshteinAtThresholds("soc_sec_id", [2]),
        cl.ExactMatch("street_number").configure(term_frequency_adjustments=True),
        cl.ExactMatch("postcode").configure(term_frequency_adjustments=True),
    ],
    "retain_intermediate_calculation_columns": True,
}

linker = Linker(df, settings, database_api=DuckDBAPI())

In [8]:
from splink.blocking_rule_library import block_on
from splink.blocking_rule_library import BlockingRuleCreator
deterministic_rules = [
    block_on("soc_sec_id"),
    block_on("given_name", "surname", "date_of_birth"),
    "l.given_name = r.surname and l.surname = r.given_name and l.date_of_birth = r.date_of_birth"
]

# TODO: cumulative_num_comparisons_from_blocking_rules_chart should accept
# BlockingRuleCreator objects
deterministic_rules = [
    br.get_blocking_rule("duckdb") if isinstance(br, BlockingRuleCreator) else br
    for br in deterministic_rules
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.9)

Probability two random records match is estimated to be  0.000528.
This means that amongst all possible pairwise record comparisons, one in 1,893.56 are expected to match.  With 12,497,500 total possible comparisons, we expect a total of around 6,600.00 matching pairs


In [9]:
linker.estimate_u_using_random_sampling(max_pairs=1e6)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - given_name (no m values are trained).
    - surname (no m values are trained).
    - date_of_birth (no m values are trained).
    - soc_sec_id (no m values are trained).
    - street_number (no m values are trained).
    - postcode (no m values are trained).


In [10]:
em_blocking_rule_1 = block_on("date_of_birth")
session_dob = linker.estimate_parameters_using_expectation_maximisation(em_blocking_rule_1)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."date_of_birth" = r."date_of_birth"

Parameter estimates will be made for the following comparison(s):
    - given_name
    - surname
    - soc_sec_id
    - street_number
    - postcode

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - date_of_birth



Iteration 1: Largest change in params was -0.373 in the m_probability of surname, level `Exact match on surname`
Iteration 2: Largest change in params was 0.0205 in the m_probability of given_name, level `All other comparisons`
Iteration 3: Largest change in params was -0.000859 in the m_probability of postcode, level `Exact match on postcode`
Iteration 4: Largest change in params was -4.54e-05 in the m_probability of postcode, level `Exact match on postcode`

EM converged after 4 iterations

Your model is not yet fully trained. Missing estimates for:
    - date_of_birth (no m values are trained).


In [11]:
em_blocking_rule_2 = block_on("postcode")
session_postcode = linker.estimate_parameters_using_expectation_maximisation(em_blocking_rule_2)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."postcode" = r."postcode"

Parameter estimates will be made for the following comparison(s):
    - given_name
    - surname
    - date_of_birth
    - soc_sec_id
    - street_number

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - postcode



Iteration 1: Largest change in params was 0.0662 in probability_two_random_records_match
Iteration 2: Largest change in params was -0.00143 in the m_probability of date_of_birth, level `Exact match on date_of_birth`
Iteration 3: Largest change in params was -3.97e-05 in the m_probability of date_of_birth, level `Exact match on date_of_birth`

EM converged after 3 iterations

Your model is fully trained. All comparisons have at least one estimate for their m and u values


In [12]:
linker.match_weights_chart()

alt.VConcatChart(...)

In [13]:
results = linker.predict(threshold_match_probability=0.2)

In [14]:
linker.roc_chart_from_labels_column("cluster", match_weight_round_to_nearest=0.1)

alt.Chart(...)

In [15]:
pred_errors_df = linker.prediction_errors_from_labels_column("cluster").as_pandas_dataframe()
len(pred_errors_df)
pred_errors_df.head()

,clerical_match_score,found_by_blocking_rules,match_weight,match_probability,rec_id_l,rec_id_r,given_name_l,given_name_r,gamma_given_name,tf_given_name_l,...,postcode_l,postcode_r,gamma_postcode,tf_postcode_l,tf_postcode_r,bf_postcode,bf_tf_adj_postcode,cluster_l,cluster_r,match_key
0,1.0,False,-26.923375,7.856996e-09,rec-829-dup-0,rec-829-dup-2,wilde,kyra,0,0.0002,...,3859,3595,0,0.0004,0.0006,0.230198,1.0,rec-829,rec-829,5
1,1.0,False,-22.231421,2.030838e-07,rec-993-dup-1,rec-993-dup-3,westbrook,jake,0,0.0004,...,2704,2074,0,0.0002,0.0014,0.230198,1.0,rec-993,rec-993,5
2,1.0,False,-18.837305,2.135036e-06,rec-829-dup-0,rec-829-dup-1,wilde,kyra,0,0.0002,...,3859,3889,0,0.0004,0.0002,0.230198,1.0,rec-829,rec-829,5
3,1.0,True,-18.804514,2.184120e-06,rec-721-dup-0,rec-721-dup-1,mikhaili,elly,0,0.0008,...,4806,4860,0,0.0008,0.0014,0.230198,1.0,rec-721,rec-721,2
4,1.0,True,-13.960824,6.271133e-05,rec-12-dup-2,rec-12-dup-4,barnaby,siggins,0,0.0008,...,5126,5612,0,0.0002,0.0002,0.230198,1.0,rec-12,rec-12,3


In [16]:
records = linker.prediction_errors_from_labels_column("cluster").as_record_dict(limit=10)
linker.waterfall_chart(records)

alt.LayerChart(...)